In [81]:
import numpy as np
from numpy import array,matrix, linalg,full
from datetime import date, timedelta
import pandas as pd

In [82]:
#read in the data generated from our rmd file
games = pd.read_csv("NCAA 2024.csv")
teamNames = pd.DataFrame(pd.read_csv("Teams.csv"))

games.head()

,Unnamed: 0,game_id,team_short_display_name,team_id,team_abbreviation,team_home_away,opponent_team_id,opponent_team_display_name,opponent_team_abbreviation,game_date,team_score,winning_score,Result
0,1,401625690,Wisconsin,275,WIS,away,356,Illinois Fighting Illini,ILL,2024-03-17,87,93,Loss
1,2,401625690,Illinois,356,ILL,home,275,Wisconsin Badgers,WIS,2024-03-17,93,93,Win
2,3,401625472,Temple,218,TEM,away,5,UAB Blazers,UAB,2024-03-17,69,85,Loss
3,4,401625472,UAB,5,UAB,home,218,Temple Owls,TEM,2024-03-17,85,85,Win
4,5,401625459,Florida,57,FLA,away,2,Auburn Tigers,AUB,2024-03-17,67,86,Loss


In [83]:
teams = set(games.team_id)
print("We're going to be looking at data across {} teams in NCAA".format(len(teams)))

#The Colley method uses matrix theory and linear algebra to solve the equation 
    # Cr = b
b = full((len(teams),1),0) 

#Creating the square matrix defaulting values to 0
    #This will be about a 700x700 matrix of teams' data
c = full((len(teams),len(teams)),0)

We're going to be looking at data across 717 teams in NCAA


In [84]:
#creating reference dictionary to build out our C-matrix
     #Ensures team_id ties to an index in the matrix
    
teams_dict = {}
for i,v in enumerate(sorted(teams)):
    teams_dict[v] =i

In [85]:
for x in range(len(c)):
    c[x][x] = 2

In [86]:
#We're going to build the c-matrix and prep to build the b-matrix to solve for the ranking coefficient

colley = {}
TestColley = {}

for i in range(len(games)):
    
    weighting = 1
    
    #Let's give away wins greater weight
    if games.iloc[i].team_home_away == "away" and games.iloc[i].Result == "Win":
        weighting += .5
    
    
    # Subtract 1 to [team_id][opponent_id] for each iteration 
        #This will be be reflected on the inverse side of the matrix when we come across the second iteration of the game
    c[teams_dict[games.iloc[i].team_id]][teams_dict[games.iloc[i].opponent_team_id]] -= weighting
    
    # Add 1 to the diagonal for each iteration where the team appears
    c[teams_dict[games.iloc[i].team_id]][teams_dict[games.iloc[i].team_id]] += weighting
    
    #We're going to keep track of Wins and Losses using this method
        #This will help when switching between Laplace ranking and Colley ranking if we so desire
    if teams_dict[games.iloc[i].team_id] not in colley:
        TestColley[teams_dict[games.iloc[i].team_id]] = {"W":0, "L":0}
    
    if games.iloc[i].Result == "Win":
        TestColley[teams_dict[games.iloc[i].team_id]]["W"] += weighting
    else:
        TestColley[teams_dict[games.iloc[i].team_id]]["L"] += weighting

In [87]:
for key in teams_dict:
    b[teams_dict[key]] = 1 + (TestColley[teams_dict[key]]["W"]-TestColley[teams_dict[key]]["L"])*.5

In [88]:
ranks = linalg.solve(c,b).tolist()

In [89]:
rank_tuples = []

#Create pairings between the power ranking and team_id
for i in range(len(ranks)):
    rank_tuples.append((ranks[i][0], list(teams_dict.keys())[list(teams_dict.values()).index(i)] ))
    
    
#If you want to look at the raw data

#sorted(rank_tuples, reverse = True)

In [48]:
#Put it in a pandas DataFrame and add a rank column
    #Given the fact we're loading the sorted rankings into the DataFrame, we could also just use the index as the ranking

Colley_coef = pd.DataFrame(sorted(rank_tuples, reverse = True), columns = ["Colley Coefficient", "team_id"] )
Colley_coef["rank"] = Colley_coef["Colley Coefficient"].rank(ascending = False)

,Colley Coefficient,team_id,rank
0,1.262164,2509,1.0
1,1.254382,248,2.0
2,1.244442,41,3.0
3,1.227096,153,4.0
4,1.220962,2633,5.0
...,...,...,...
712,0.362467,111170,713.0
713,0.362467,126821,714.0
714,0.348194,42,715.0
715,0.346195,126910,716.0


In [49]:
CM = pd.merge(Colley_coef, teamNames, left_on= "team_id", right_on = "team_id", how = "inner")

In [50]:
CM

,Colley Coefficient,team_id,rank,Unnamed: 0,team_display_name
0,1.262164,2509,1.0,480,Purdue
1,1.254382,248,2.0,241,Houston
2,1.244442,41,3.0,628,UConn
3,1.227096,153,4.0,414,North Carolina
4,1.220962,2633,5.0,591,Tennessee
...,...,...,...,...,...
711,0.362467,111170,713.0,197,FRANCISCAN
712,0.362467,126821,714.0,466,Pit Greensburg
713,0.348194,42,715.0,228,Hartford
714,0.346195,126910,716.0,460,Penn State-NK


In [22]:
CM.to_excel("Colley Rankings.xlsx")